In [1]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\InvitadoLaptop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\InvitadoLaptop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import spotipy
import pandas as pd
import tqdm
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
CLIENT_ID = 'e89a20fa9f524fbe91a34ed52c346ce5'
CLIENT_SECRET = '5b776dd228714f9cad9716a07a6d6a25'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
artistas_list = ['Grupo 5', 'Ariana Grande', 'Slipknot', 'Amelie lens', 'Paramore']

album_data = []

In [9]:
for artista_nombre in tqdm.tqdm(artistas_list, desc='Obteniendo álbumes de artistas:'):
    resultados = sp.search(q=f'artist:{artista_nombre}', type='album', limit=10)

    for album in resultados['albums']['items']:
        nombre_album = album['name']
        fecha_lanzamiento = album['release_date']
        id_album = album['id']

        artistas = [artist['name'] for artist in album['artists']]
        artista_principal = artistas[0] if artistas else "Artista no disponible"

        genero = "Género no disponible"  
        if album['artists']:
           artista_id = album['artists'][0]['id']
           artista_info = sp.artist(artista_id)
           generos_artista = artista_info.get('genres', [])
           if generos_artista:
              genero = generos_artista[0]  

    año_lanzamiento = fecha_lanzamiento[:4]  

    album_info = {
                 "Artista": artista_principal,
                 "Género": genero,
                 "Tipo": "Álbum",  
                 "Nombre": nombre_album,
                 "Año de lanzamiento": año_lanzamiento,
                 "ID": id_album,
                 "Todos los artistas": ", ".join(artistas)
                 }

    album_data.append(album_info)

    
df_albums = pd.DataFrame(album_data)

df_albums

Obteniendo álbumes de artistas:: 100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


,Artista,Género,Tipo,Nombre,Año de lanzamiento,ID,Todos los artistas
0,Grupo Maravilla,cumbia sonidera,Álbum,5 de Abril,2005,3dXbyP6YApDEGSzfBQOCrm,Grupo Maravilla
1,Ariana Grande,pop,Álbum,the boy is mine (Remix),2024,5zesPlZ3PXnxQ8HWzQ9idI,"Ariana Grande, Brandy, Monica"
2,Bendrick_Slipknot,Género no disponible,Álbum,Do It for Eggy,2023,4UnuG6frFmJYJ0ykL6nrTK,Bendrick_Slipknot
3,Amelie Lens,techno,Álbum,Hypnotized,2019,4aYZyDTf6sBTfPWuFpRrgU,Amelie Lens
4,Paramore Squads,Género no disponible,Álbum,Machine's Lament,2025,2OQxsQidDsZzYWuzzpdGAI,Paramore Squads
5,Grupo Maravilla,cumbia sonidera,Álbum,5 de Abril,2005,3dXbyP6YApDEGSzfBQOCrm,Grupo Maravilla
6,Ariana Grande,pop,Álbum,the boy is mine (Remix),2024,5zesPlZ3PXnxQ8HWzQ9idI,"Ariana Grande, Brandy, Monica"
7,Bendrick_Slipknot,Género no disponible,Álbum,Do It for Eggy,2023,4UnuG6frFmJYJ0ykL6nrTK,Bendrick_Slipknot
8,Amelie Lens,techno,Álbum,Hypnotized,2019,4aYZyDTf6sBTfPWuFpRrgU,Amelie Lens
9,Paramore Squads,Género no disponible,Álbum,Machine's Lament,2025,2OQxsQidDsZzYWuzzpdGAI,Paramore Squads
